In [1]:
import pandas as pd
import numpy as np

In [2]:
data_set=pd.read_csv('data\medical_conversations.csv')

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\GUNATEJA\AppData\Local\Temp\ipykernel_1932\3296619104.py:1: SyntaxWarning: invalid escape sequence '\m'
  data_set=pd.read_csv('data\medical_conversations.csv')


In [3]:
data_set.head()

,conversations,disease
0,User: I’ve been sneezing a lot today and my no...,allergy
1,User: I’ve developed a rash after eating some ...,allergy
2,"User: My eyes are swollen and itchy, and I can...",allergy
3,User: I’ve been getting headaches and a stuffy...,allergy
4,"User: Every time I eat nuts, my mouth itches. ...",allergy


In [5]:
data_set["disease"].value_counts()

disease
allergy                            40
bronchial asthma                   40
common cold                        40
fungal infection                   40
acne                               40
gastroesophageal reflux disease    40
hypertension                       40
dimorphic hemorrhoids              40
typhoid                            40
chicken pox                        40
cervical spondylosis               40
peptic ulcer disease               40
arthritis                          40
urinary tract infection            40
migraine                           40
jaundice                           40
pneumonia                          40
psoriasis                          40
drug reaction                      40
diabetes                           40
varicose veins                     40
impetigo                           40
malaria                            40
dengue                             40
Name: count, dtype: int64

In [6]:
data_set.shape

(960, 2)

In [7]:
data_set.isnull().sum()

conversations    0
disease          0
dtype: int64

In [8]:
data_set["conversations"][0]

'User: I’ve been sneezing a lot today and my nose feels congested. </s> Bot: That sounds like it could be an allergy. Do you know what might be triggering it? </s> User: I’m not sure. Maybe pollen? </s> Bot: Pollen is a common allergen. Have you had any other symptoms, like itchy eyes or a sore throat? </s> User: Yes, my eyes are itchy too. </s> Bot: Sounds like classic hay fever. Over-the-counter antihistamines could help relieve the symptoms. </s>'

In [9]:
x=data_set["conversations"]
y=data_set["disease"]

In [10]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)
y_one_hot = to_categorical(y_encoded, num_classes=num_classes)

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(x)
X_tokenized = tokenizer.texts_to_sequences(x)

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 100
X_padded = pad_sequences(X_tokenized, maxlen=max_length, padding='post')

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_one_hot, test_size=0.2, random_state=42)

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense

model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=max_length), 
    LSTM(128, return_sequences=False), 
    Dropout(0.5),  
    Dense(64, activation='relu'),  
    Dropout(0.5), 
    Dense(num_classes, activation='softmax') 
])

C:\Users\GUNATEJA\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 4s 76ms/step - accuracy: 0.0426 - loss: 3.1669 - val_accuracy: 0.0938 - val_loss: 3.0182
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.0888 - loss: 3.0619 - val_accuracy: 0.0781 - val_loss: 2.8623
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.0845 - loss: 2.9346 - val_accuracy: 0.1406 - val_loss: 2.9022
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.1035 - loss: 2.8974 - val_accuracy: 0.1562 - val_loss: 2.7516
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.1128 - loss: 2.7703 - val_accuracy: 0.1927 - val_loss: 2.5917
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.1499 - loss: 2.6186 - val_accuracy: 0.1562 - val_loss: 2.5152
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.1785 - loss: 2.5041 - val_accuracy: 0.2031 - val_loss: 2.3204
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 67ms/step - accuracy: 0.2072 - loss: 2.3055 - val_accuracy: 0.1979 - v

In [17]:
model.save("lstm_text_classification_model.h5")

In [18]:
import pickle
with open('tokenizer7.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)

In [19]:
def predict_disease(input_text):
    input_text = input_text.lower()
    tokenized = tokenizer.texts_to_sequences([input_text])
    padded = pad_sequences(tokenized, maxlen=max_length, padding='post')
    prediction = model.predict(padded)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]

# Example usage
user_input = "I have fever and body aches"
predicted_disease = predict_disease(user_input)
print(f"Predicted Disease: {predicted_disease}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
Predicted Disease: impetigo
